In [1]:
# third party
from faker import Faker
from fixtures import *

In [2]:
manager = TestEventManager()
manager.reset_test_state()

In [3]:
USERS_CREATED = "users_created"
MOCK_READABLE = "mock_readable"

In [4]:
fake = Faker()

In [5]:
admin = make_admin()
admin

TestUser(name='Eduardo Edwards', email='info@openmined.org', password='changethis', role=<ServiceRole.ADMIN: 128>, server_cache=None)

In [7]:
async with AsyncWaitForEvent(manager, MOCK_READABLE, retry_secs=1, timeout_secs=10):
    print("Test Complete")
    result = manager.get_event(MOCK_READABLE)
    print(result)
    assert result

Test Complete
None


AssertionError: 

In [ ]:
server = make_server(admin)

In [ ]:
root_client = admin.client(server)

In [ ]:
dataset_name = fake.name()
dataset = create_dataset(name=dataset_name)

In [ ]:
upload_dataset(root_client, dataset)

In [ ]:
users = [make_user() for i in range(2)]
users

In [ ]:
def create_users(root_client, manager, users):
    for test_user in users:
        create_user(root_client, test_user)
    manager.register_event(USERS_CREATED)

In [ ]:
def user_can_read_mock_dataset(server, manager, user, dataset_name):
    print("waiting ", USERS_CREATED)
    with WaitForEvent(manager, USERS_CREATED, retry_secs=1):
        print("logging in user")
        user_client = user.client(server)
        print("getting dataset", dataset_name)
        mock = user_client.api.services.dataset[dataset_name].assets[0].mock
        df = trade_flow_df_mock(trade_flow_df())
        assert df.equals(mock)
        manager.register_event(MOCK_READABLE)

In [ ]:
user = users[0]
user

In [ ]:
asyncit(
    user_can_read_mock_dataset,
    server=server,
    manager=manager,
    user=user,
    dataset_name=dataset_name,
)

In [ ]:
asyncit(create_users, root_client=root_client, manager=manager, users=users)

In [ ]:
with WaitForEvent(manager, MOCK_READABLE, retry_secs=1, timeout_secs=10):
    print("Test Complete")
    result = manager.get_event(MOCK_READABLE)
    print(result)
    assert result